### Basic forward-design example
Demonstrate the basic input format that can be used for forward design.
Designs can have fused regions and holes, specified by the `fusedPts` and `holePts`, respectively.

The provided design doesn't do anything interesting...

In [ ]:
import sys; sys.path.append('..')
import inflation, mesh, sheet_meshing
from tri_mesh_viewer import TriMeshViewer as Viewer
import triangulation
import numpy as np
import utils, py_newton_optimizer, benchmark

In [ ]:
V, E = mesh.load_raw('data/ForwardDesign/example.obj')
fusedPts = list(np.loadtxt('data/ForwardDesign/example_fusedPts.txt').reshape((-1, 2)))
holePts  = list(np.loadtxt('data/ForwardDesign/example_holePts.txt').reshape((-1, 2)))

In [ ]:
m, iwv, iwbv = sheet_meshing.forward_design_mesh(V, E, fusedPts, holePts, np.prod(utils.bbox_dims(V)[0:2]) / 1e4)
isheet = inflation.InflatableSheet(m, iwv)

In [ ]:
import visualization
visualization.plot_2d_mesh(m, pointList=np.where(iwv))

In [ ]:
viewer = Viewer(isheet, wireframe=True)
viewer.show()

In [ ]:
import time
benchmark.reset()
isheet.setUseTensionFieldEnergy(True)
isheet.setUseHessianProjectedEnergy(False)
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 500
isheet.pressure = 1
framerate = 5 # Update every 5 iterations
def cb(it):
    if it % framerate == 0:
        viewer.update()
cr = inflation.inflation_newton(isheet, isheet.rigidMotionPinVars, opts, callback=cb)
benchmark.report()